In [25]:
import numpy as np
import pandas as pd

In [26]:
data = pd.read_csv("TASK2_dataset.csv")
data_encoded = pd.get_dummies(data, columns=['keyword_1', 'keyword_2', 'keyword_3'])
data_encoded = data_encoded.drop(columns=['song_id'])#column song id not required


genre_mapping = {genre: idx for idx, genre in enumerate(data_encoded['genre'].unique())}  # Convert genre to numerical labels
data_encoded['genre'] = data_encoded['genre'].map(genre_mapping)

In [27]:
X = data_encoded.drop(columns=['genre']).values #features and target
y = data_encoded['genre'].values
num_classes = len(genre_mapping)
y_one_hot = np.eye(num_classes)[y]  #Convert labels to one-hot encoding


softmax function for multi-class logistic regression

In [28]:
def softmax(z):
    z = np.array(z, dtype=np.float64)  # Ensure input is an array
    if z.ndim == 0:  # If z is a scalar, convert it to a 1D array
        z = np.array([z])
    exp_z = np.exp(z - np.max(z, axis=-1, keepdims=True))  # Stability trick
    return exp_z / np.sum(exp_z, axis=-1, keepdims=True)

creating softmaxregression model

In [29]:
class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = y.shape[1]

        # Initialize weights and bias
        self.weights = np.zeros((num_features, num_classes))
        self.bias = np.zeros((1, num_classes))

        # Gradient Descent
        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = softmax(linear_model)

            # Compute gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y, axis=0, keepdims=True)

            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = softmax(linear_model)
        return np.argmax(y_pred, axis=1) 

In [30]:
model = SoftmaxRegression(learning_rate=0.01, num_iterations=1000)
model.fit(X, y_one_hot)

In [31]:
def predict_genre(keyword_1, keyword_2, keyword_3):
    # Create a DataFrame for the input keywords
    input_data = pd.DataFrame({
        'keyword_1': [keyword_1],
        'keyword_2': [keyword_2],
        'keyword_3': [keyword_3]
    })

    # One-hot encode the input keywords
    input_encoded = pd.get_dummies(input_data)

    # Ensure input_encoded has the same columns as data_encoded
    input_encoded = input_encoded.reindex(columns=data_encoded.drop(columns=['genre']).columns, fill_value=0)

    # Predict the genre
    prediction = model.predict(input_encoded.values)
    predicted_genre = list(genre_mapping.keys())[prediction[0]]  # Map back to genre name
    return predicted_genre

In [32]:
def compute_accuracy(model, X, y):
    predictions = model.predict(X)
    y_true = np.argmax(y, axis=1)  # Convert one-hot back to label indices
    accuracy = np.mean(predictions == y_true) * 100  # Convert to percentage
    return accuracy

accuracy = compute_accuracy(model, X, y_one_hot)
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 74.15%


In [33]:
print(predict_genre('guitar', 'emotional', 'distorted'))  
print(predict_genre('synth', 'mellow', 'distorted'))  
print(predict_genre('piano', 'calm', 'slow'))

rock
hip-hop
classical
